In [10]:
# data manipulations
import numpy as np
import pandas as pd
# progress bar
from tqdm.notebook import tqdm_notebook
# audio analysis
import librosa

In [3]:
to_load = pd.read_csv("to_load.csv")
to_load

,path,label,samplingrate,nframes,ms
0,./Labelled/Bleatings/evt_000_000_000681_210415...,Bleatings,16000,156016,9751.0
1,./Labelled/Bleatings/evt_000_000_000682_210415...,Bleatings,16000,99040,6190.0
2,./Labelled/Bleatings/evt_000_000_000683_210415...,Bleatings,16000,97152,6072.0
3,./Labelled/Bleatings/evt_000_000_000684_210415...,Bleatings,16000,97696,6106.0
4,./Labelled/Bleatings/evt_000_000_002434_210416...,Bleatings,16000,164224,10264.0
...,...,...,...,...,...
822,./Labelled/Others/evt_003_003_069816_210609_06...,Others,16000,228400,14275.0
823,./Labelled/Others/evt_003_003_069817_210609_06...,Others,16000,97568,6098.0
824,./Labelled/Others/evt_003_003_069818_210609_06...,Others,16000,119280,7455.0
825,./Labelled/Others/evt_003_003_069819_210609_06...,Others,16000,96448,6028.0


# Extract features set

In [5]:
# shuffle the data
to_load = to_load.sample(frac=1)

In [11]:
features = []
for element in tqdm_notebook(to_load.values.tolist()[:50]):
    features_vector = []
    filename = element[0]
    y, sr = librosa.load(filename)
    
    # add filename for later indexing the dataframe
    features_vector.append(filename)
    
    rms = librosa.feature.rms(y=y)
    features_vector.append(np.mean(rms))
    #chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    #spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    #spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    #rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    features_vector.append(np.mean(zcr))
    
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    features_vector = features_vector + [np.mean(e) for e in mfccs]
    
    #to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
    #for e in mfcc:
    #    to_append += f' {np.mean(e)}'
    
    # add the label
    features_vector.append(element[1])
    features.append(features_vector)

  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
features_df = pd.DataFrame(features, columns=["filename", "rms", "zcr"] + [f'mfcc{i}' for i in range(20)]+ ["label"])
features_df = features_df.set_index("filename")
features_df.head()

,rms,zcr,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,label
filename,,,,,,,,,,,,,,,,,,,,,
./Labelled/Bleatings/evt_000_003_017370_210422_143739.wav,0.016555,0.098719,-333.899994,142.983856,-56.197685,41.681320,-15.505000,9.609718,7.344662,10.028235,...,5.601384,4.752415,-4.012461,3.964277,-0.381156,10.751761,8.088605,4.815539,-0.730275,Bleatings
./Labelled/Others/evt_000_001_050725_210507_041404.wav,0.006705,0.064619,-410.253906,160.915268,-40.434196,52.212326,-19.761183,26.582207,-6.959056,13.494112,...,-0.789553,5.086392,-5.798947,7.198698,-9.410315,6.736906,-3.345610,4.026073,-0.839583,Others
./Labelled/Others/evt_000_004_056196_210512_095729.wav,0.022746,0.076490,-305.385803,136.546478,-26.544491,24.138884,2.732224,4.278108,3.278859,2.297472,...,-2.268005,1.848497,-1.533101,0.513620,-0.386419,4.270723,0.889709,2.118828,3.440112,Others
./Labelled/Others/evt_000_003_069975_210516_172141.wav,0.012991,0.092632,-322.414490,164.939301,-68.396179,43.906757,-21.451475,26.462368,-17.275246,16.139709,...,-3.744617,9.959677,-7.539350,6.872898,-4.783088,8.239529,-5.825597,2.630365,0.786140,Others
./Labelled/Others/evt_001_003_028392_210429_190724.wav,0.007446,0.089268,-362.616333,142.717361,-47.316021,55.047962,-21.027283,26.120560,-12.984117,9.336222,...,-3.544557,7.074082,-6.766791,11.192875,-7.160723,6.033654,-2.298451,3.237531,-0.823168,Others


In [13]:
features_df.to_csv("sample.csv")